# prepare_protein — Run a batch protein preperation in parallel

This notebook shows how to load a large amount of data and execute runs on them in parallel

# 0) Complete example
See the [sample notebook](/Quickstarts/batch_run-protein-prep-sample.ipynb) for a quick demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
import asyncio
from glob import glob
from datetime import datetime
from pathlib import Path

from pdbtools import (
    pdb_fetch,
    pdb_delhetatm,
    pdb_selchain,
    pdb_rplresname,
    pdb_keepcoord,
    pdb_selresname,
)
import py3Dmol

import rush

## 1.1) Configuration

In [ ]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "batch-prep"]
WORK_DIR = Path.home() / "qdx" / "rush-py-batch-prep"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

TARGET = "BULLET_2"

In [ ]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

Ensure your workdir exists

In [ ]:
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 1.2) Build your client

Initialize our rush client and fetch available module paths.

In [ ]:
os.environ["RUSH_TOKEN"] = YOUR_TOKEN

# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    batch_tags=TAGS, workspace=WORK_DIR
)

In [ ]:
# fetch datafiles
for pdb in ["1B39", "4QXI", "8FSU"]:
    complex = list(pdb_fetch.fetch_structure(pdb))
    protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))

    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb}_protein.pdb", "w") as f:
        for l in protein:
            f.write(str(l))

In [ ]:
help(client.prepare_protein)

# 2) Prepare each protein

In [ ]:
proteins = map(lambda x: Path(x), glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")))

protein_outputs = []

for protein_path in proteins:
    print(protein_path)
    name = protein_path.stem
    (prepped_protein_qdxf, prepped_protein_pdb) = await client.prepare_protein(
        protein_path, None, None, tags=[name], target=TARGET
    )
    protein_outputs.append((name, prepped_protein_qdxf, prepped_protein_pdb))

protein_outputs

# 3) Report progress
This will show the status of all of your runs

In [ ]:
protein_outputs[0][1].download()

In [ ]:
status = await client.status(group_by="path")
print(f"{'Module':<32} | {'Status':<32} | Count")
print("-" * 75)
for module, (status, path, count) in status.items():
    print(f"{path:<32} | {status:<32} | {count:>5}")

# 4) Download Results
This will retrieve results for your completed module_instances

In [ ]:
protein_outputs[0][1]

In [ ]:
await asyncio.gather(
    *[
        output[1].download(
            filename=f"protein_{output[0]}_prepared.qdxf.json", overwrite=True
        )
        for output in protein_outputs
    ]
    + [
        output[2].download(
            filename=f"protein_{output[0]}_prepared.pdb", overwrite=True
        )
        for output in protein_outputs
    ],
)

# 5) Visualize Results

In [ ]:
view = py3Dmol.view(viewergrid=(1, 3))

prepared_proteins = map(
    lambda x: Path(x), glob(str(client.workspace / "objects" / "*.pdb"))
)

for i, file in enumerate(prepared_proteins):
    with open(file, "r") as f:
        view.addModel(f.read(), "pdb", viewer=(0, i))
        view.setStyle({"cartoon": {"color": "spectrum"}}, viewer=(0, i))
        view.zoomTo(viewer=(0, i))
view.show()

# 6) Check failures
This will retrieve failed runs in your workspace history

In [ ]:
for instance_id, (status, name, count) in (await client.status()).items():
    if status.value == "FAILED":
        async for log_page in client.logs(instance_id, "stderr"):
            for log in log_page:
                print(log)